# 第2章 函数

## 第14条 尽量用异常来表示特殊情况，而不要返回None

### 要点
#### 1.用None这个返回值来表示特殊意义的函数，很容易使调用者犯错，因为None和0及空字符串之类的值，在条件表达式里都会评估为False
#### 2.函数在遇到特殊情况时，应该抛出异常，而不要返回None。调用者看到该函数的文档中所描述的异常之后，应该就会编写相应的代码来处理它们了

例如，编写辅助函数，计算两数相除的商

In [1]:
def divide(a, b):
    try:
        return a/b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

In [4]:
x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Results is %.1f' % result)

Results is 2.5


## 第15条 了解如何在闭包里使用外围作用域中的变量

### 要点
#### 1.对于定义在某作用域内的闭包来说，它可以引用这些作用域中的变量
#### 2.使用默认方式对闭包内的变量赋值，不会影响外围作用域中的同名变量
#### 3.在Python3中，程序可以在闭包内用nonlocal语句来修饰某个名称，使该闭包能够修改外围作用域中的同名变量
#### 4.除了那种比较简单的函数，尽量不要用nonlocal语句

假如有一份列表，其中的元素都是数字，现在要对其排序，但排序时，要把出现在某个群组内的数字，放在群组外的那些数字之前。还应该返回一个值，用来表示用户界面里是否出现了优先级较高的元件。

In [5]:
def sort_priority(number, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return(1, x)
    number.sort(key=helper)
    return found
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
is_found = sort_priority(numbers, group)
print(is_found)
print(numbers)

True
[2, 3, 5, 7, 1, 4, 6, 8]


## 第16条 考虑用生成器来改写直接返回列表的函数

### 要点
#### 1.使用生成器比把收集到的结果放入列表里返回给调用者更加清晰
#### 2.由生成器函数所返回的那个迭代器，可以把生成器函数体中，传给yield表达式的那些值，逐次产生出来
#### 3.无论输入量多大，生成器都能产生一系列输出，因为这些输入量和输出量，都不会影响它在执行时所耗的内存

假设我们要查出字符串中每个词的首字母,在整个字符串里的位置，把结果存在一个列表里。有两个缺点，一个是代码拥挤，重要的代码只有一半；第二个是在返回前，要先把所有结果都存在列表里面，如果输入量非常大，可能耗尽内存而崩溃

In [1]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index+1)
    return result

In [2]:
address = 'Four score and seven years ago.'
result = index_words(address)
print(result[:3])

[0, 5, 11]


下面的这个生成器函数，会产生和刚才那个函数相同的效果，且更清晰。且用生成器改写的版本，可以应对任意长度的输入数据。

In [4]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index+1
result = list(index_words_iter(address))
print(result)

[0, 5, 11, 15, 21, 27]


## 第17条 在参数上面迭代时，要多加小心
### 要点
#### 1.函数在输入的参数上面多次迭代时要当心：如果参数是迭代器，那么可能会导致奇怪的行为并错失某些值
#### 2.Python的迭代器协议，描述了容器和迭代器应该如何与iter和next内置函数、for循环及相关表达式相互配合
#### 3.把__iter__方法实现为生成器，即可定义自己的容器类型
#### 4.想判断某个值是迭代器还是容器，可以拿该值为参数，两次调用iter函数，若结果相同，则是迭代器，调用内置的next函数，即可令该迭代器前进一步

## 第18条 用数量可变的位置参数减少视觉杂讯
### 要点
#### 1.在def语句中使用*args，即可令函数接受数量可变的位置参数
#### 2.调用函数时，可以采用*操作符，把序列中的元素当成位置参数，传给该函数
#### 3.对生成器使用*操作符，可能导致程序耗尽内存并奔溃
#### 4.在已经接受*args参数的函数上面继续添加位置参数，可能会产生难以排查的bug

定义Log函数，把某些调试信息打印出来

In [5]:
def log(message, *values):
    if not values:
        print(message)
    else:
        value_str = ', '.join(str(x) for x in values)
        print('%s: %s' % (message, value_str))
log('My numbers are', 1, 2)
log('Hi there')

My numbers are: 1, 2
Hi there


可以给列表前面加上*操作符，这样Python会把这个列表里的元素视为位置参数

In [6]:
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

Favorite colors: 7, 33, 99


## 第19条 用关键字参数来表达可选的行为
### 要点
#### 1.函数参数可以按位置或关键字来指定
#### 2.只使用位置参数来调用函数，可能会导致这些参数值的含义不够明确，而关键字参数则能够阐明每个参数的意图
#### 3.给函数添加新的行为时，可以使用带默认值的关键字参数，以便与原有的函数调用代码保持兼容
#### 4.可选的关键字参数，总是应该以关键字形式来指定，而不应该以位置参数的形式来指定

## 第20条 用None和文档字符串来描述具有动态默认值的参数
### 要点
#### 1.参数的默认值，只会在程序加载模块并读到本函数的定义时评估一次。对于{}或[]等动态的值，这可能会导致奇怪的行为
#### 2.对于以动态值作为实际默认值的关键字参数来说，应该把形式上的默认值写为None，并在函数的文档字符串里面描述该默认值所对应的实际行为

例如，在打印日志消息的时候，要把相关事件的记录时间也标注在这条信息中。要想正确实现动态默认值，习惯上是把默认值设为None，
并在文档字符串里面把None所对应的实际行为描述出来。编写函数代码时，如果发现该参数的值是None，那就将其设为实际的默认值。
下面是正确的写法：

In [5]:
from time import sleep
from datetime import datetime

def log(message, when=None): #如果写when=datetime.now()，是错误的
    '''Log a message with a timestamp.
    
    Args:
        message: Message to print
        when: datetime of when the message occurred.
            Defaults to the present time.
    '''
    when = datetime.now() if when is None else when
    print('%s: %s' % (when, message))
log('Hi there!')
sleep(0.1)
log('Hi again!')

2017-07-09 22:33:24.710984: Hi there!
2017-07-09 22:33:24.811555: Hi again!


## 第21条 用只能以关键字形式指定的参数来确保代码清晰
### 要点
#### 1.关键字参数能够使函数调用的意图更加明确
#### 2.对于各参数之间很容易混淆的函数，可以声明只能以关键字形式指定的参数，以确保调用者必须通过关键字来指定它们，对于接受多个Boolean标志的函数，更应该这样做
#### 3.在编写函数时，Python3有明确的语法来定义这种只能以关键字形式指定的参数
#### 4.Python2的函数可以接受**kwargs参数，并手工抛出TypeError异常，以便模拟只能以关键字形式来指定的参数

参数列表里的*号，标志着位置参数就此终结，之后的那些参数，都只能以关键字形式来指定

In [6]:
def safe_division(number, divisor, *, ignore_overflow=False, ignore_zero_division=False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [7]:
safe_division(1, 0, ignore_zero_division=True)

inf

In [8]:
try:
    safe_division(1, 0)
except ZeroDivisionError:
    pass